# The Multinomial Distribution

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

## Probability distributions in `scipy.stats`

In [ ]:
help(sp.stats.multinomial)

We'll be working primarily with two methods of `scipy.stats.multinomial`: `pmf()`, and `rvs()`,
as well as a small set of parameters with consistent naming between the two methods: `x`, `n`, `p`, and `size`.

To maintain clarity, I'll always try to use the same number of dimensions for the inputs and outputs of
these functions, but it's worth noticing that distributions in 
`scipy.stats` do some tricky stuff with dimensions just to mess with you.

- `x` : The observed counts in all $k$ categories.
- `n` : The total number of samples taken.
- `p` : The expected fraction of each category.
- `size` : the number of trials: repeating the random draws with the same parameters.

`.rvs()` draws values from the distribution.

In [ ]:
n = np.array(100)
p = np.array([0.1, 0.5, 0.4])
size = 10

x = sp.stats.multinomial.rvs(n, p, size)
print(x)

# EXERCISE: Mess with the parameters and see what happens.

`.rvs()` calculates the probability of drawing a value.

In [ ]:
n = 30
p = [1/3, 1/3, 1/3]
x_diff = [15, 15, 0]
x_same = [30, 0, 0]

print(sp.stats.multinomial.pmf(x_diff, n, p))
print(sp.stats.multinomial.pmf(x_same, n, p))

# EXERCISE: Mess with the data and see what happens.

Like, `R`, `scipy.stats` takes pains to be as consistent as possible across methods and distributions.
Fortunately that means you usually only have to learn weird quirks of the API once.

EXERCISE: What happens when `np.sum(p) != 1.0`?

## DNA composition as a multinomial model

Let's think about how to translate a real system into a simple probabilistic model.

A single strand of DNA is composed of $n$ positions, each of which may be in one of
four states: $\{\mathrm{A}, \mathrm{C}, \mathrm{G}, \mathrm{T}\}$.
While evolutionary selection potentially acting on this sequence means that any one position
is not randomly assigned a base, our ignorance of this fact means that at
larger scales the composition of bases is akin to repeatedly flipping a 4-sided coin
... I mean rolling a 4-sided die ... or whatever.

How do we relate the ideas in this abstract biological system to the parameters of the
multinomial?

EXERCISE: What do each of the variables represent?

```python
n = None  # ???
p = None  # ???
size = None  # ???
x = None  # ???
```

EXERCISE: What scenario in the context above does the following model describe?

```python
sp.stats.multinomial.rvs(100, [1/4, 1/4, 1/4, 1/4], size=1)
```

## The test statistics

Much of frequentist statistics is about understanding how "surprising" a result is.
This is exactly how a p-value should be interpretted: "we would get this 'extreme'
of a result only $p$ portion of the time under the null hypothesis.

The first step in this process is deciding what we mean by "extreme".
We'll need to choose a metric (formula) that calculates
an extreme-ness score for our data, with larger values being more extreme/surprising.
This formula is called a "test statistic" and for multivariate data
(like our nucleotide base counts) it also has the benefit of
being 1-dimensional rather than $k$-dimensional.

You should think of test statistics as distances, they are (usually?) in the set
$[0, \inf]$ with larger values being more surprising.
Sometimes we can parameterize our test statistic with a hypothesis
(usually the null), but sometimes we can't and the hypothesis is
implicit.

For multinomial-like data, the sum of relative squared differences from the expectation
is a natural (whatever that means) test statistic.

In [ ]:
def sum_of_relative_squared_differences(x, n, p):
    expect = p * n
    return np.sum((expect - x)**2 / expect)

In [ ]:
n = 2
p = np.array([1/4, 1/4, 1/4, 1/4])
x_diff = np.array([1, 1, 0, 0])
x_same = np.array([2, 0, 0, 0])

print(sum_of_relative_squared_differences(x_diff, n, p))
print(sum_of_relative_squared_differences(x_same, n, p))

# EXERCISE: Play around with _which_ bases we hit under each scenario
# (e.g. what if we hit both C's instead of both A's),
# and see how it affects the test statistic.

Our statistic is telling us that it's more "extreme" for both positions
to have the same base than it is for them to have different bases.

## The null hypothesis

Before we go any further, we should be sure to state what we mean by "surprising".
This depends on what we expect.
In a hand-wavy sort of way, what we expect is often described as the "null
hypothesis" in frequentist statistics.  (I say hand-wavy, because there
much of the time we don't _really_ expect the null, but that's a very different
discussion.)

The test statistic that we've selected for the multinomial (`sum_of_relative_squared_differences`)
is pretty spiffy because we can parameterize it with whatever null hypothesis we want.
Usually that's uniform across all of the options, $p_i = p_j$ for all $i$ and all $j$.

In [ ]:
n = 4
p = np.array([1/4, 1/4, 1/4, 1/4])
x_diff = np.array([1, 1, 1, 1])
x_same = np.array([2, 1, 1, 0])

print(sum_of_relative_squared_differences(x_diff, n, p))
print(sum_of_relative_squared_differences(x_same, n, p))

# EXERCISE: What happens to the test statistics under each scenario when you change
# the expected proportion, `p`?

## Statistical testing

Okay, so we've concluded that under the uniform model described above,
it's surprising to get lot's more of one base than the others.

The next question is "how surprising?"

For a p-value threshold of 0.05, what we are saying is:

> if our test statistic is more extreme than what we'd expect 95% of
> the time when our null hypothesis is true, then we decide it probably
> isn't and "reject" it.

So what we need to test our null hypothesis is a way to
calculate the probability of seeing a more extreme value under the null.
Notice that we're talking about our test statistic as though _it_ is being
drawn from a random distribution.

And it is;
since it's a function of our data which is drawn from a probability distribution,
the test statistic is itself drawn from a probability distribution.

But what distribution?

Sometimes we know, and this is one of those cases:
under the null hypothesis and given some assumptions
the sum of relative squared differences
is well approximated by draws from the $\chi^2$ distribution under the
(with $k - 1$ degrees of freedom).

In [ ]:
x_obs = np.array([25, 25, 25, 25])

p_null = np.array([1/4, 1/4, 1/4, 1/4])
n = np.sum(x_obs)
df = p.shape[0] - 1

statistic = sum_of_relative_squared_differences(x_obs, n, p_null)
# The probability of a more extreme value is the complement of the probability of a
# less extreme value.
pvalue = 1 - sp.stats.chi2.cdf(statistic, df)
print(pvalue)

# EXERCISE: What happens to the p-value as you change the data, `x`?

## Simulating data, statistics, p-values

So the multinomial is handy because we've got a natural test statistic
with a well understood distribution,
but what do we do when we don't know the exact distribution of our test statistic?

There are plenty of things we _can_ do, but maybe the most obvious is to
simulation data from the null hypothesis and compare our observed test
statistic to the simulated one.

In [ ]:
p_null = np.array([1/4, 1/4, 1/4, 1/4])
n = 1000
size = 10000

x_sim = sp.stats.multinomial.rvs(n, p_null, size=size)

statistic_sim = np.apply_along_axis(
    lambda y: sum_of_relative_squared_differences(y, n, p_null),
    axis=1,
    arr=x_sim
                                    )

plt.hist(statistic_sim, bins=100, density=True)
# Add the chi2 PDF for comparison
ss = np.linspace(0, 20, num=1000)
df = p.shape[0] - 1
plt.xlabel('SSD')
plt.plot(ss, sp.stats.chi2.pdf(ss, df))

We can then compare our observed test statistic to arrive at a p-value.

In [ ]:
x_obs = np.array([290, 210, 250, 250])
assert sum(x_obs) == n
statistic_obs = sum_of_relative_squared_differences(x_obs, n, p_null)

plt.hist(statistic_sim, bins=100, density=True)
plt.axvline(statistic_obs, lw=1, color='k')
plt.xlabel('SSD')
print((statistic_sim > statistic_obs).mean())

By the same logic that we concluded the test statistic is randomly distributed,
the p-value is also randomly distributed due to its relationship with the random
data.

Let's simulate a bunch of different data and see what this p-value distribution
looks like.

In [ ]:
p_null = np.array([1/4, 1/4, 1/4, 1/4])
n = 1000
size = 10000

x_sim = sp.stats.multinomial.rvs(n, p_null, size=size)
df = p.shape[0] - 1

statistic_sim = np.apply_along_axis(
    lambda y: sum_of_relative_squared_differences(y, n, p_null),
    axis=1,
    arr=x_sim
                                    )
# I'm going back to the analytical distribution here, but you could
# sub in the null distribution produced through simulation.
pvalue_sim = 1 - sp.stats.chi2.cdf(statistic_sim, df)

plt.hist(pvalue_sim, bins=20)
plt.axvline(0.05, lw=1, color='r', linestyle='--')
plt.xlabel('p')
print((pvalue_sim < 0.05).mean())

# EXERCISE: Why is the p-value distributed as it is under the null?

# EXERCISE: Why is the value of `n` set to such a large value?
# What happens when it's set to 100? 10?

Notice that about 5% of the time our p < 0.05 ... under the null!

These are false positives; $p < \alpha$
a fraction of the time equal to $\alpha$.

But what happens when the simulated data doesn't come from the null?

In [ ]:
p_null = np.array([1/4, 1/4, 1/4, 1/4])
p_sim = np.array([0.27, 0.23, 0.25, 0.25])
n = 1000
size = 10000

x_sim = sp.stats.multinomial.rvs(n, p_sim, size=size)
df = p.shape[0] - 1

statistic_sim = np.apply_along_axis(
    lambda y: sum_of_relative_squared_differences(y, n, p_null),
    axis=1,
    arr=x_sim
                                    )
pvalue_sim = 1 - sp.stats.chi2.cdf(statistic_sim, df)

plt.hist(pvalue_sim, bins=20)
plt.axvline(0.05, lw=1, color='r', linestyle='--')
plt.xlabel('p')
print((pvalue_sim < 0.05).mean())

# EXERCISE: What happens when you mess with the parameters?

Clearly we get far more p-values < 0.05, we therefore reject the null
a larger fraction of the time, and since we _know_ that the null
is wrong (our data are simulated from a non-uniform distribution),
our "hits" are true positives.

On the other hand, we also have plenty of trials where we did not
reject the null hypothesis.  These are now false negatives.

The fraction of trials that reject the null hypothesis is our
study's "power".

EXERCISE: What parameters does this power depend on?

## Power analysis

Let's put it all together into a power analysis function.

In [ ]:
def power(p, n, alpha=0.05, p_null=None, nsim=1000):
    k = p.shape[0]
    if p_null is None:
        p_null = np.ones_like(p) / k
        
    x_sim = sp.stats.multinomial.rvs(n, p, size=nsim)
    df = k - 1
    statistic_sim = np.apply_along_axis(
        lambda y: sum_of_relative_squared_differences(y, n, p_null),
        axis=1,
        arr=x_sim
                                        )
    # If using an emprical null distribution, you'd probably
    # want to pre-calculte the CDF, since you'll often call
    # power() multiple times.
    pvalue_sim = 1 - sp.stats.chi2.cdf(statistic_sim, df)
    return (pvalue_sim < alpha).mean()

And let's run it on the same data as before.

In [ ]:
p_null = np.array([1/4, 1/4, 1/4, 1/4])
p_sim = np.array([0.27, 0.23, 0.25, 0.25])
n = 1000

print(power(p_sim, n))

# EXERCISE: What happens each time you re-run the function?

While there are sometimes analytical solutions for the power, here we're using simulation.
As the study design (and therefore the data distribution) gets more complicated, simulation
will quickly become the only option.

Conditional on an expected fractions of each base
we can now calculate our power at varying sample sizes,
$n$.

In [ ]:
p_sim = np.array([0.27, 0.23, 0.25, 0.25])
alpha = 0.05
nsim = 1000

nn = np.logspace(1, 4, num=50).astype(int)
p_reject = [power(p_sim, n, alpha=alpha, nsim=nsim) for n in nn]

plt.scatter(nn, p_reject, marker='.')
plt.axhline(alpha, linestyle='--', lw=1, color='k')
plt.axhline(0.8, linestyle='--', lw=1, color='k')
plt.xlabel('n')
plt.ylabel('power')
plt.xscale('log')

What we've found is that if we want a 80% chance of rejecting the null hypothesis
we need a much larger sample size.

Also notice the diminishing returns at high values of $n$.

## Conclusion

That's it.

We've done it.

We now know how to use simulation to calculate
the power of our multinomial test for a given
value of the underlying parameter $p$, with a given
sample size $n$ and p-value threshold $\alpha$.

For other scenarios the procedure is the same:

1. Pick a data generating distribution
2. Pick a test statistic measuring the "extremeness" of the data: $\phi(\cdot)$
3. Figure out (or simulate) the distribution of this test statistic under the null in order to calculate how surprising observed data is (a p-value): $1 - \mathrm{CDF}_{\mathrm{\phi(x_{null})}}(\phi(x))$
4. Simulate the distribution of this test statistic under hypothetical parameters
4. Calculate the power given your choice of parameters: $\mathrm{Pr}(1 - \mathrm{CDF}_{\mathrm{null}}(\phi(x)) < \alpha)$

For many simple distributions these steps have already been worked out for you,
but often they have not been.
In that case, simulation is a powerful tool.

As a result, the challenge is usually picking "realistic" parameters for
the data generating process, rather than calculating the power itself.